In [1]:
data Heap a =
    Empty |
    Tree {
        _rank :: !Int,
        _value :: !a,
        _left :: Heap a,
        _right :: Heap a
    } deriving (Show)

In [8]:
merge :: Ord a => Heap a -> Heap a -> Heap a
merge h Empty = h
merge Empty h = h
merge h1@(Tree _ v1 l1 r1) h2@(Tree _ v2 l2 r2)
    | v1 < v2 = makeT v1 l1 (merge r1 h2)
    | otherwise = makeT v2 l2 (merge r2 h1)
    
rank :: Heap a -> Int
rank Empty = 0
rank (Tree r _ _ _) = r

makeT :: a -> Heap a -> Heap a -> Heap a
makeT v h1 h2 =
    let
        r1 = rank h1
        r2 = rank h2
    in
    if r1 < r2 
    then Tree (r1+1) v h2 h1
    else Tree (r2+1) v h1 h2
    
insert :: Ord a => a -> Heap a -> Heap a
insert a  = merge (Tree 1 a Empty Empty)

findMin :: Heap a -> a
findMin (Tree _ x _ _) = x

deleteMin :: Ord a => Heap a -> Heap a
deleteMin (Tree _ _ l r) = merge l r

singleton :: a -> Heap a
singleton x = Tree 1 x Empty Empty

In [7]:
--Exercise 3.2
directInsert :: Ord a => a -> Heap a -> Heap a
directInsert x Empty = Tree 1 x Empty Empty
directInsert x h@(Tree r v lt rt) 
    | x < v = Tree 1 x h Empty
    | otherwise = Tree r v (directInsert x lt) rt

In [25]:
--Exercise 3.3
import Data.List

fromList :: Ord a => [a] -> Heap a
fromList lst = fromList' (map singleton lst)
    where 
        fromList' [t] = t
        fromList' (x : y : xs) = fromList' $ pair (x : y: xs)
        pair (x : y : xs) = merge x y : pair xs
        pair xs = xs

In [21]:
--Exercise 3.4 (b)
data WHeap a =
    Empty |
    Tree {
        _size :: !Int,
        _value :: !a,
        _left :: WHeap a,
        _right :: WHeap a
    } deriving (Show)

In [22]:
--Exercise 3.4 (c)
size :: WHeap a -> Int
size Tree s _ _ _ = s

wMerge :: Ord a => WHeap a -> WHeap a -> WHeap a
wMerge wh Empty = wh
wMerge Empty wh = wh
wMerge wh1@(Tree s1 v1 lt1 rt1) wh2@(Tree s2 v2 lt2 rt2) 
    | v1 <= v2, size lt1 >= size rt1 + s2 = Tree (s1+s2) v1 wh1 (wMerge rt1 wh2)
    | v1 <= v2 = Tree (s1+s2) v1 (wMerge rt1 wh2) lt1
    | otherwise = wMerge wh2 wh1

In [40]:
data BTree a = 
    Node {
        _rank :: !Int,
        _value :: !a,
        _children :: [BTree a]
    } deriving (Show)
    
link :: Ord a => BTree a -> BTree a -> BTree a
link t1@(Node r v1 c1) t2@(Node _ v2 c2) 
    | v1 < v2 = Node (r+1) v1 (t2 : c1)
    | otherwise = Node (r+1) v2 (t1 : c2)
    
singleton :: a -> BTree a
singleton x = Node 0 x []
    
type BHeap a = [BTree a]

rank :: BTree a -> Int
rank (Node r _ _) = r

root :: BTree a -> a
root (Node _ x _) = x

children :: BTree a -> [BTree a]
children (Node _ _ c) = c

insTree :: Ord a => BTree a -> BHeap a -> BHeap a
insTree t [] = [t]
insTree t ts@(t' : ts')
    | rank t < rank t' = t : ts
    | otherwise = insTree (link t t') ts' --Always assume rank(t) <= rank(t') !!
    
insert :: Ord a => a -> BHeap a -> BHeap a
insert x = insTree (singleton x)

merge :: Ord a => BHeap a -> BHeap a -> BHeap a
merge h [] = h
merge [] h = h
merge h1@(t1 : ts1) h2@(t2 : ts2) 
    | rank t1 < rank t2 = t1 : merge ts1 h2
    | rank t2 < rank t1 = t2 : merge h1 ts2
    | otherwise = link t1 t2 : merge ts1 ts2
    
removeMinTree :: Ord a => BHeap a -> (BTree a, BHeap a)
removeMinTree [t] = (t, [])
removeMinTree (t : ts) = let
    (t', ts') = removeMinTree ts
    in
    if root t <= root t' 
    then (t, ts)
    else (t', t : ts')
    
findMin :: Ord a => BHeap a -> a
findMin = root . fst . removeMinTree

deleteMin :: Ord a => BHeap a -> BHeap a
deleteMin h = let
    (t, ts) = removeMinTree h
    in
    merge (reverse . children $ t) ts

In [42]:
--Exercise 3.5
directFindMin :: Ord a => BHeap a -> a
directFindMin [t] = root t
directFindMin h@(t : ts) = let
    v = root t 
    v' = directFindMin ts
    in
    if v < v' then v else v'